<a href="https://colab.research.google.com/github/somwrks/AI-Solar-Panel/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self Rotatory Solar Panel

This Project utilizes various vision models to train on planet sun labelled datasets and perform validation checks to determine which model performs the best and converts the best model to tinyML Model to make it usable by microcontroller

## Installing Dependencies

In [ ]:
%pip install --upgrade pip


In [2]:
%pip install  tensorflow[and-cuda] tensorflow-datasets roboflow ultralytics torch torchvision matplotlib tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.7/704.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.5/197.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

## Importing Libraries

In [44]:
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
from tensorflow.keras import mixed_precision
from roboflow import Roboflow
import yaml
import os
from PIL import Image


In [45]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN custom operations


In [46]:
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.test.is_gpu_available())


TensorFlow version: 2.17.1
GPU Available: True


In [47]:
# List available GPUs
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

if len(physical_devices) > 0:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)

    tf.config.set_visible_devices(physical_devices[0], 'GPU')

    # Enable mixed precision training

    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)

    print("GPU optimization settings applied successfully")
else:
    print("No GPU available. Running on CPU.")

Num GPUs Available:  1
GPU optimization settings applied successfully


## Loading Datasets

In [54]:
import os
import yaml
from roboflow import Roboflow
from PIL import Image

# Download both datasets
rf = Roboflow(api_key="j1wL1N5IgwOIiKW4RYuo")

# Dataset 1

project1 = rf.workspace("yassine-pzpt7").project("sun-tracking-photovoltaic-panel")
version1 = project1.version(2)
dataset1 = version1.download("yolov8")

# Dataset 2
project2 = rf.workspace("1009727588-qq-com").project("sun-nxvfz")
dataset2 = project2.version(2).download("yolov8")

# Dataset 3
project3 = rf.workspace("rik-tjduw").project("sun-tracking-555mn")
version3 = project3.version(4)
dataset3 = version3.download("yolov8")

# Prepare dataset paths
dataset1_path = dataset1.location
dataset2_path = dataset2.location
dataset3_path = dataset3.location

def resize_images(directory, target_resolution=(640, 640)):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(image_extensions):
                image_path = os.path.join(root, file)
                try:
                    with Image.open(image_path) as img:
                        img_resized = img.resize(target_resolution, Image.LANCZOS)
                        img_resized.save(image_path)
                except Exception as e:
                    print(f"Error resizing image {image_path}: {e}")

# Function to count images in a directory
def count_images(directory):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    return len([f for f in os.listdir(directory) if f.lower().endswith(image_extensions)])

# Function to combine datasets

def combine_datasets(*dataset_paths, combined_path):
    for split in ['train', 'valid', 'test']:
        os.makedirs(os.path.join(combined_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(combined_path, split, 'labels'), exist_ok=True)

        for dataset_path in dataset_paths:
            images_src = os.path.join(dataset_path, split, 'images')
            labels_src = os.path.join(dataset_path, split, 'labels')

            images_dst = os.path.join(combined_path, split, 'images')
            labels_dst = os.path.join(combined_path, split, 'labels')

            if os.path.exists(images_src):
                for file in os.listdir(images_src):
                    shutil.copy2(os.path.join(images_src, file), images_dst)
            if os.path.exists(labels_src):
                for file in os.listdir(labels_src):
                    shutil.copy2(os.path.join(labels_src, file), labels_dst)


# Combine datasets
combined_path = './combined_dataset'
combine_datasets(dataset1_path, dataset2_path, dataset3_path, combined_path=combined_path)

# Resize images in the combined dataset
for split in ['train', 'valid', 'test']:
    images_dir = os.path.join(combined_path, split, 'images')
    resize_images(images_dir)

print("Images resized to target resolution.")


# Count images in each dataset
train_count = len(os.listdir(os.path.join(combined_path, 'train', 'images')))
valid_count = len(os.listdir(os.path.join(combined_path, 'valid', 'images')))
test_count = len(os.listdir(os.path.join(combined_path, 'test', 'images')))
num_classes = len(set([f.split('_')[0] for f in os.listdir(os.path.join(combined_path, 'train', 'labels'))]))

print(f"Number of images in training set: {train_count}")
print(f"Number of images in validation set: {valid_count}")
print(f"Number of images in test set: {test_count}")
print(f"Total number of images: {train_count + valid_count + test_count}")

# Create YAML file for YOLOv5
yaml_content = {
    'train': os.path.join(combined_path, 'train'),
    'val': os.path.join(combined_path, 'valid'),
    'test': os.path.join(combined_path, 'test'),
    'nc': len(os.listdir(os.path.join(combined_path, 'train', 'labels'))),
    'names': [f'class_{i}' for i in range(len(os.listdir(os.path.join(combined_path, 'train', 'labels'))))],
    'image_counts': {
        'train': train_count,
        'val': valid_count,
        'test': test_count
    }
}


yaml_content = {
    'train': os.path.join(combined_path, 'train'),
    'val': os.path.join(combined_path, 'valid'),
    'test': os.path.join(combined_path, 'test'),
    'nc': num_classes,
    'names': [f'class_{i}' for i in range(num_classes)],
}

yaml_path = os.path.join(combined_path, 'dataset.yaml')
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)

print(f"Dataset YAML file created at: {yaml_path}")


loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
Images resized to target resolution.
Number of images in training set: 1135
Number of images in validation set: 97
Number of images in test set: 52
Total number of images: 1284
Dataset YAML file created at: ./combined_dataset/dataset.yaml


## Setting Up Models


### 2. YoloV5 (LibRT)

In [55]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
yolo_model = YOLO('yolov8n.pt')

full_yaml_path = os.path.abspath(yaml_path)
yolo_results = yolo_model.train(data=full_yaml_path, epochs=100, imgsz=640)


Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/combined_dataset/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning /content/combined_dataset/train/labels... 1135 images, 214 backgrounds, 0 corrupt: 100%|██████████| 1135/1135 [00:00<00:00, 1954.73it/s]

train: New cache created: /content/combined_dataset/train/labels.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/combined_dataset/valid/labels... 97 images, 19 backgrounds, 0 corrupt: 100%|██████████| 97/97 [00:00<00:00, 892.09it/s]

...

                   all         97         78      0.817      0.744      0.777      0.361

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     23/100      2.79G      1.502       2.08      1.339         14        640: 100%|██████████| 71/71 [00:24<00:00,  2.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.07it/s]

                   all         97         78      0.804      0.821      0.779      0.416



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     24/100       2.8G      1.512      2.077      1.355         18        640:  45%|████▌     | 32/71 [00:10<00:19,  1.97it/s]

In [ ]:
# Evaluate YOLOv5
yolo_metrics = yolo_model.val()

print("YOLOv5 mAP:", yolo_metrics.results_dict['metrics/mAP50-95(B)'])


## Testing Models

In [ ]:
# tbd

## Saving Model

In [ ]:
model.save('sun_tracker_model.h5')
